In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install squarify
!pip install seaborn

In [ ]:
# İMPORT İŞLEMLERİ
import seaborn as sns
import squarify
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt

# <p style="background-color:#EFCD59;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;">e-Ticaret Satın Alma &  Müşteri Davranışı Analizi Projesi </p>

**7 AYIN VERİSİNDEN ÖRNEKLEM ÇEKİP BİRLEŞTİRME İŞLEMLERİ**

*Purchase değişkenine ağırlık verilmiştir.*


data_list = [oct_19, nov_19, dec_19, jan_20, feb_20, mar_20, apr_20]

for data in data_list:

    data['event_time'] = pd.to_datetime(data['event_time'], utc=True).dt.tz_convert('Europe/Istanbul')
    
    data.loc[data['event_type'] == 'purchase', 'is_purchase'] = 50
    
    data.loc[data['event_type'] == 'cart', 'is_purchase'] = 1
    
    data.loc[data['event_type'] == 'view', 'is_purchase'] = 1
    

oct_sample = oct_19.sample(n=250000, weights='is_purchase')

nov_sample = nov_19.sample(n=250000, weights='is_purchase')

dec_sample = dec_19.sample(n=250000, weights='is_purchase')

jan_sample = jan_20.sample(n=250000, weights='is_purchase')

feb_sample = feb_20.sample(n=250000, weights='is_purchase')

mar_sample = mar_20.sample(n=250000, weights='is_purchase')

apr_sample = apr_20.sample(n=250000, weights='is_purchase')

df_all = pd.concat([oct_sample, nov_sample, dec_sample, jan_sample, feb_sample, mar_sample, apr_sample], axis=0)

In [ ]:
df = pd.read_csv("/kaggle/input/ecommerce-crm-data/crmdata.csv")
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

**DEĞİŞKENLER**

* event_time : işlemin yapıldığı tarih & saat
* event_type : yapılan işlemin türü, purchase (satın alma), cart (sepete ekleme), view (ürüne göz atma)
* product_id : ürünün id'si
* category_id : ürünün kategorisini temsil eden id
* category_code : ürünün ana/alt kategorisi
* brand : marka ismi
* price : ürünün fiyatı
* user_id : kullanıcı id'si
* user_session : kullanıcı oturum id'si

<a id="3"></a>
# <p style="background-color:#6D50FF;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> KEŞİFÇİ VERİ ANALİZİ </p>

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Columns #####################")
    print(dataframe.columns)

check_df(df)

In [ ]:
df["user_id"].nunique()

In [ ]:
# GÖRÜNTÜLENME/SEPETE EKLEME/SATIN ALMA DAĞILIMI

ratio = pd.DataFrame(df["event_type"].value_counts().reset_index())
ratio.columns = ["event_type","count"]
ratio["ratio"] = ratio["count"] * 100 / df.shape[0]
ratio

In [ ]:
# GÖRÜNTÜLENME/SEPETE EKLEME/SATIN ALMA DAĞILIMI PASTA GRAFİĞİ
labels = ['view', 'purchase','cart']
size = df['event_type'].value_counts()
colors = ["lightcoral","lightskyblue","yellowgreen"]
explode = [0, 0.1,0.1]

plt.rcParams['figure.figsize'] = (8, 8)
plt.pie(size, colors = colors, explode = explode, labels = labels, shadow = True, autopct = '%.2f%%')
plt.title('Event_Type', fontsize = 20)
plt.axis('off')
plt.legend()
plt.show()

In [ ]:
# EN ÇOK SATAN 15 MARKA
purchase = df.loc[df['event_type'] == 'purchase']
top_sellers = purchase.groupby('brand')["brand"].agg(['count']).sort_values('count', ascending=False)
top_sellers = top_sellers.head(15)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_sellers.index, y=top_sellers['count'], palette="Pastel2")
plt.title("En Çok Satan 15 Marka")
plt.xlabel("Marka")
plt.ylabel("Satış Sayısı")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# EN ÇOK GÖRÜNTÜLENEN 15 MARKA
view = df.loc[df['event_type'] == 'view']
top_view_brand = view.groupby('brand')["brand"].agg(['count']).sort_values('count', ascending=False)
top_view_brand = top_view_brand.head(15)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_view_brand.index, y=top_sellers['count'], palette="Pastel1")
plt.title("En Çok Görüntülenen 15 Marka")
plt.xlabel("Marka")
plt.ylabel("Görüntülenme Sayısı")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# EN ÇOK SATAN KATEGORİLER

purchase_df = df.loc[df['event_type'] == 'purchase']
top_categories_purchase = purchase_df.groupby('category_code')["category_code"].agg(['count']).sort_values('count', ascending=False)
top_categories_purchase = top_categories_purchase.reset_index()
top_categories_purchase.head(15)

In [ ]:
# EN ÇOK SATAN 15 KATEGORİNİN GRAFİĞİ 

plt.figure(figsize=(12, 6))
plt.bar(top_categories_purchase['category_code'][:15], top_categories_purchase['count'][:15], color='lightsalmon')
plt.xlabel('Kategori Kodu')
plt.ylabel('Satın Alınma Sayısı')
plt.title('En Çok Satan Kategoriler (İlk 15)')
plt.xticks(rotation=45, ha='right')  # X ekseni etiketlerini 45 derece sağa döndürün
plt.tight_layout()
plt.show()

In [ ]:
# EN ÇOK GÖRÜNTÜLENEN KATEGORİLER

view_df = df.loc[df['event_type'] == 'view']
top_categories_view = view_df.groupby('category_code')["category_code"].agg(['count']).sort_values('count', ascending=False)
top_categories_view = top_categories_view.reset_index()
top_categories_view.head(15)

**CATEGORY_CODE DEĞİŞKENİNDEKİ KATEGORİLERİ ANA KATEGORİ VE ALT KATEGORİ OLARAK AYIRIP YENİ DEĞİŞKENLER ÜRETTİK**

In [ ]:
#CATEGORY_CODE SÜTUNUNDAKİ DEĞERLERİ NOKTA (.) KARAKTERİNE GÖRE AYIRMA FONKSİYONLARI TANIMLIYORUZ.

def extract_category_level_1(x):
    if pd.notna(x):
        return x.split('.')[0]
    else:
        return np.nan
    
def extract_category_level_2(x):
    if pd.notna(x):
        return x.split('.')[1]
    else:
        return np.nan
    
# TANIMLADIĞIMIZ FONKSİYONLAR İLE YENİ SÜTUNLAR OLUŞTURUYORUZ (ANA KATEGORİ VE ALT KATEGORİ)

df["category_code_level1"] = df["category_code"].apply(extract_category_level_1)
df["category_code_level2"] = df["category_code"].apply(extract_category_level_2)

# ANA KATEGORİLERİN SATIŞ SAYISI GRAFİĞİNİ OLUŞTURALIM

df[df["event_type"]=="purchase"].groupby("category_code_level1")["category_code_level1"].count().sort_values(ascending=False).plot(kind="bar")
plt.show()

In [ ]:
# EVENT_TİME DEĞİŞKENİNİN TİPİNİ DATETİME'A ÇEVİRME VE TARİH/SAAT DEĞİŞKENLERİ OLUŞTURMA İŞLEMLERİ

df['event_time'] = pd.to_datetime(df['event_time'])
df["event_date"]= df["event_time"].dt.date
df['event_date'] = pd.to_datetime(df['event_date'])
df['event_hour'] = df['event_time'].dt.hour
df["event_day"] = df['event_time'].dt.day_name()
df['month_name'] = df['event_date'].dt.strftime('%B')

In [ ]:
df.head()

In [ ]:
#SATIN ALMA İŞLEMLERİ DATAFRAME'İ
purchase = df.loc[df['event_type'] == 'purchase']

In [ ]:
# AYLARA GÖRE SATIN ALMA İŞLEMLERİ ANALİZİ

purchase_by_month = purchase[['month_name','user_id']].drop_duplicates().groupby(['month_name'])['user_id'].agg(['count']).sort_values(by=["month_name"])
purchase_by_month

In [ ]:
# TARİHE GÖRE SATIN ALMA İŞLEMLERİ ANALİZİ

purchase_by_date = purchase[['event_date','user_id']].drop_duplicates().groupby(['event_date'])['user_id'].agg(['count']).sort_values(by=["event_date"])

x = pd.Series(purchase_by_date.index.values)
y = purchase_by_date['count']
plt.rcParams['figure.figsize'] = (20,8)

plt.plot(x,y)
plt.show()

In [ ]:
# SAATE GÖRE SATIN ALMA İŞLEMLERİ ANALİZİ

hourly_purchase_counts = purchase.groupby('event_hour')['event_type'].count()

plt.figure(figsize=(12, 6))
hourly_purchase_counts.plot(kind='bar', color='lightgreen')
plt.xlabel('Saat')
plt.ylabel('Satın Alma Sayısı')
plt.title('Saatlere Göre Satın Alma İşlemleri')
plt.show(block=True)

In [ ]:
# GÜNLERE GÖRE SATIN ALMA İŞLEMLERİ ANALİZİ

purchase_counts_for_days = purchase.groupby('event_day')['event_type'].count()

plt.figure(figsize=(12, 6))
purchase_counts_for_days.plot(kind='bar', color='#FFC0CB')
plt.xlabel('Saat')
plt.ylabel('Satın Alma Sayısı')
plt.title('Günlere Göre Satın Alma İşlemleri')
plt.show(block=True)

<a id="7"></a>
# <p style="background-color:#E24356;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> SAYISAL DEĞİŞKEN ANALİZİ </p>

In [ ]:
#BİR ÜRÜNÜN FİYATI 0 OLAMAZ DİYE DÜŞÜNÜP "PRİCE"IN 0'DAN BÜYÜK OLDUĞU SATIRLARI FİLTRELEDİK.
df = df[df["price"]> 0]

In [ ]:
df[["price"]].describe().T

In [ ]:
#SATIN ALMA İŞLEMLERİ DATAFRAME'İ

purchase = df.loc[df['event_type'] == 'purchase']

In [ ]:
# MARKALARIN 7 AYDA FİRMAYA GETİRDİĞİ TOPLAM PARASAL DEĞER

pd.options.display.float_format = '{:,.3f}'.format
brand_price_analysis_sum = purchase.groupby("brand")["price"].sum().sort_values(ascending=False).reset_index()

brand_price_analysis_sum.head(20)


In [ ]:
# MARKALARIN 7 AYDA FİRMAYA GETİRDİĞİ TOPLAM PARASAL DEĞER GRAFİĞİ

brand_price_analysis_sum_head20 = brand_price_analysis_sum.head(20)
plt.figure(figsize=(12, 10))
plt.bar(brand_price_analysis_sum_head20["brand"], brand_price_analysis_sum_head20["price"])
plt.xlabel('Marka')
plt.ylabel('Toplam Satış Getirisi')
plt.title('Markaların Toplam Satış Getirisi')
plt.xticks(rotation=45)  # X ekseni etiketlerini döndürmek için
plt.show()

In [ ]:
# SATIN ALINAN KATEGORİLERİN TOPLAM PARASAL DEĞER TABLOSU

cat_cod_analysis = purchase.groupby("category_code")["price"].sum().sort_values(ascending=False).reset_index()
cat_cod_analysis.head(15)


In [ ]:
# SATIN ALINAN ANA KATEGORİLERİN TOPLAM PARASAL DEĞER TABLOSU

cat_cod_1_analysis = purchase.groupby("category_code_level1")["price"].sum().sort_values(ascending=False).reset_index()
cat_cod_1_analysis


In [ ]:
# SATIN ALINAN ALT KATEGORİLERİN TOPLAM PARASAL DEĞER TABLOSU

cat_cod_2_analysis = purchase.groupby("category_code_level2")["price"].sum().sort_values(ascending=False).reset_index()
cat_cod_2_analysis.head(15)

In [ ]:
# FİRMAYA EN FAZLA PARASAL DEĞER KATAN İLK 15 MÜŞTERİ
# Ayrıntılı müşteri analizini RFM bölümünde yapacağız ama yine de basit bir müşteri satın alma analizi yapalım istedik.

user_analysis = purchase.groupby("user_id")["price"].sum().sort_values(ascending=False).reset_index()
user_analysis.head(15)

<a id="5"></a>
# <p style="background-color:#A5D204;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> KATEGORİK DEĞİŞKEN ANALİZİ </p>

In [ ]:
cat_cols = ["brand", "category_code", "category_code_level1", "category_code_level2"]

def cat_summary(dataframe, col_name, plot=False, top_n=20):
    summary = pd.DataFrame({col_name: dataframe[col_name].value_counts().head(top_n),
                            "Ratio": 100 * dataframe[col_name].value_counts(normalize=True).head(top_n)})
    print(summary)
    print("##########################################")
    if plot:
        plt.figure(figsize=(12, 6))
        plt.bar(summary.index, summary['Ratio'], color='pink')
        plt.xlabel(col_name)
        plt.ylabel('Oran (%)')
        plt.title(f'Tüm verideki {col_name} işlemleri (İlk {top_n})')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

for col in cat_cols:
    cat_summary(df, col, plot=True)

<a id="2"></a>
# <p style="background-color:#E8A84B;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> HEDEF DEĞİŞKEN ANALİZİ ⚒</p>

*Hedef değişkenimizi satın alma işlemi olarak kabul ediyoruz.*

In [ ]:
brand_analysis = df.groupby(['brand', 'event_type']).size().unstack(fill_value=0).sort_values(by = "purchase" , ascending=False)
brand_analysis = brand_analysis.reset_index()
brand_analysis.head(15)

In [ ]:
event_date_analysis = df.groupby(['month_name', 'event_type']).size().unstack(fill_value=0).sort_values(by = "purchase" , ascending=False)
event_date_analysis

In [ ]:
category_analysis = df.groupby(['category_code', 'event_type']).size().unstack(fill_value=0).sort_values(by = "purchase" , ascending=False)
category_analysis.head(20)

<a id="6"></a>
# <p style="background-color:#65DDBA;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> RFM ANALİZİ </p>

Recency: Müşterinin en son ne zaman alışveriş yaptığını gösterir. Müşterinin sıcaklığını, yeniliğini ifade eder.

Frequency : Müşterinin satın alma / işlem sıklığını gösterir.

Monetary : Müşterinin firmaya bıraktığı parasal değeri gösterir.

In [ ]:
#SATIN ALMA İŞLEMLERİ DATAFRAME'İ

purchase = df.loc[df['event_type'] == 'purchase']
purchase.head()

In [ ]:
# EN AZ 1 KERE SATIN ALMA YAPAN MÜŞTERİ SAYISI

purchase["user_id"].nunique()

In [ ]:
purchase["event_date2"] = purchase["event_date"].dt.date

In [ ]:
purchase["event_date2"].max()

In [ ]:
today_date = dt.date(2020,5,2)

In [ ]:
# MÜŞTERİLERİN RECENCY, FREQUENCY, MOMETARY DEĞERLERİNİN HESAPLANMASI

rfm_df = purchase.groupby("user_id").agg({"event_date2" : lambda x : (today_date - x.max()).days,
                                      "event_type" : "count",
                                      "price" : "sum"})
rfm_df.columns = ["recency","frequency","monetary"]

In [ ]:
rfm_df.describe().T

In [ ]:
# RF ve RFM Skorlarının Hesaplanması (standartlaştırma)

rfm_df["r_score"] = pd.qcut(rfm_df["recency"], 5 , [5,4,3,2,1])
rfm_df["f_score"] = pd.qcut(rfm_df["frequency"].rank(method="first"), 5, [1,2,3,4,5])
rfm_df["m_score"] = pd.qcut(rfm_df["monetary"], 5 , [1,2,3,4,5])
rfm_df["rf_score"] = rfm_df["r_score"].astype(str) + rfm_df["f_score"].astype(str)

In [ ]:
rfm_df.head()

In [ ]:
# RF Skorlarının Segment Olarak Tanımlanması

seg_map = {
    r"[1-2][1-2]":"hibernating",
    r"[1-2][3-4]":"at_risk",
    r"[1-2]5":"cant_loose",
    r"3[1-2]": "about_to_sleep",
    r"33": "need_attention",
    r"[3-4][4-5]":"loyal_customers",
    r"41": "promising",
    r"51": "new_customers",
    r"[4-5][2-3]":"potential_loyalists",
    r"5[4-5]":"champions"
}

rfm_df["segment"] = rfm_df["rf_score"].replace(seg_map, regex=True)

In [ ]:
rfm_df.head()

In [ ]:
#  MÜŞTERİ SEGMENT ANALİZİ

rfm_df[["segment","recency","frequency","monetary"]]. groupby("segment").agg(["mean","count"])

In [ ]:
# ÖRNEĞİN NEW_CUSTOMERS SEGMENTİNDEKİ USER_İD'LERİ KAYDEDELİM.

new_df = pd.DataFrame()

new_df["new_customer_id"] = rfm_df[rfm_df["segment"] == "new_customers"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

new_df.to_csv("new_customers.csv")